+# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [3]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [5]:
recommender = MainRecommender(data_train_lvl_1)

In [6]:
recommender.get_als_recommendations(2375, N=5)

[899624, 871756, 1044078, 8090521, 1022428]

In [7]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [8]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 937292, 934399, 12781666]

In [9]:
recommender.get_similar_users_recommendation(2375, N=5)

[974265, 1012801, 921406, 12263857, 1134296]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [18]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()

result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1 = result_lvl_1.query('user_id in @train_users')
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [11]:
# result_lvl_1 = result_lvl_1.head(255)

#### Зависимость от способа генерации кандидатов

In [19]:
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=50) for i in list(result_lvl_1['user_id'])]

In [82]:
#result_lvl_1

In [20]:
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=50) for i in list(result_lvl_1['user_id'])]

In [21]:
result_lvl_1['simr_it_rec'] = [recommender.get_similar_items_recommendation(i, N=50) for i in list(result_lvl_1['user_id'])]

In [23]:
# result_lvl_1['simr_us_rec'] = [recommender.get_similar_users_recommendation(i, N=5) for i in list(result_lvl_1['user_id'])]

In [24]:
# your_code
result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

1.3079084086113886

In [25]:
result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

1.8201887674891033

In [26]:
result_lvl_1.apply(lambda x: recall_at_k(x['simr_it_rec'], x['actual']), axis=1).mean() * 100

0.5930034504279519

#### Зависимость от k

get_own_recommendations

In [61]:
%%time
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=2) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

Wall time: 9.64 s


1.0044194943022828

In [62]:
%%time
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=3) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

Wall time: 10.2 s


1.4157798576125875

In [43]:
%%time
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=4) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

1.6289420719598082

In [44]:
%%time
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=5) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

1.8201887674891033

In [59]:
%%time
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=100) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

Wall time: 10.3 s


1.8201887674891033

In [31]:
%%time
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=200) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

1.8201887674891033

In [60]:
%%time
result_lvl_1['own_rec'] = [recommender.get_own_recommendations(i, N=300) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['own_rec'], x['actual']), axis=1).mean() * 100

Wall time: 10.6 s


1.8201887674891033

get_als_recommendations

In [47]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=2) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.7383722346755862

In [48]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=3) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.9950999425217446

In [49]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=4) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

1.1670629487190272

In [50]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=5) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

1.3079084086113886

In [33]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=20) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

1.3079084086113886

In [34]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=100) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

1.3079084086113886

In [35]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=200) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

1.3079084086113886

In [36]:
%%time
result_lvl_1['als_rec'] = [recommender.get_als_recommendations(i, N=300) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

1.3079084086113886

get_similar_items_recommendation

In [51]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=2) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.28930172040208324

In [52]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=3) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.3999641913419112

In [53]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=4) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.5024539783669515

In [54]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=5) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.5930034504279519

In [37]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=20) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.5930034504279519

In [38]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=100) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.5930034504279519

In [39]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=200) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.5930034504279519

In [40]:
%%time
result_lvl_1['als_rec'] = [recommender.get_similar_items_recommendation(i, N=300) for i in list(result_lvl_1['user_id'])]

result_lvl_1.apply(lambda x: recall_at_k(x['als_rec'], x['actual']), axis=1).mean() * 100

0.5930034504279519

### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [63]:
# your_code
data_train_lvl_2.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0
2107469,2021,40618753059,594,856060,1,1.77,443,-0.09,101,86,0.0,0.0
2107470,2021,40618753059,594,869344,1,1.67,443,-0.22,101,86,0.0,0.0
2107471,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,0.0,0.0


In [65]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=5))

In [66]:
users_lvl_2.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263]"
1,2021,"[950935, 1119454, 835578, 863762, 1019142]"


In [67]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1105426,1
0,2070,1097350,1
0,2070,879194,1
0,2070,948640,1


In [68]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [69]:
targets_lvl_2['target'].mean()

0.2867504835589942

In [70]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [71]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [77]:
# user_features['average_sum'] = []
# user_features[user_features['user_id'] == 7]['average_sum'] = data_train_lvl_2[data_train_lvl_2['user_id'] == 7]['sales_value'].sum()

In [73]:
# генерация признаков:
# для пользователя:
# for i in user_features["user_id"]:
#     user_features[user_features['user_id'] == i]['average_sum'] = data_train_lvl_2[data_train_lvl_2['user_id'] == i]['sales_value'].sum()

<ipython-input-73-b5f4e7f4f80e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_features[user_features['user_id'] == i]['average_sum'] = data_train_lvl_2[data_train_lvl_2['user_id'] == i]['sales_value'].sum()


### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 